In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F

from einops import rearrange
#from torchsummary import summary

import sys
sys.path.insert(0, '../helpers')
# sys.path.insert(0,'/home/labs/rudich/sagima/helpers')
from plotting import *
from validate import *
from models import *
from data_handlers import *
from training import *
from metrics import *
from explore import *
from torchsummary import *
from visual_helpers_2 import *

#from visual_helpers import *

import itertools

import json

#taken from the model building section - 
import pandas as pd
pd.set_option('max_colwidth', 800)


import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import xgboost as xg
from statsmodels.tools.eval_measures import rmse
import scipy 
from statsmodels.tsa.stattools import grangercausalitytests
import torch.optim as optim
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.fft import fft, fftfreq
import folium

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
import cartopy.feature


import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio


import ast
from scipy.spatial.distance import cdist

from tqdm import tqdm


/home/labs/rudich/sagima/.conda/envs/sagi_base/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/labs/rudich/sagima/.conda/envs/sagi_base/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
df_cord = pd.read_csv("/home/labs/rudich/sagima/TOAR/coordinates_pm.csv")
df_cord = df_cord.rename(columns={df_cord.columns[0]:'station_id'})


In [3]:
df_pm0 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_0.csv",index_col=0).dropna(axis=1,how='all')
df_pm1 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_1.csv",index_col=0).dropna(axis=1,how='all')
df_pm2 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_2.csv",index_col=0).dropna(axis=1,how='all')
df_pm3 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_3.csv",index_col=0).dropna(axis=1,how='all')
df_pm4 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_4.csv",index_col=0).dropna(axis=1,how='all')
df_pm5 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_5.csv",index_col=0).dropna(axis=1,how='all')
df_pm6 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_6.csv",index_col=0).dropna(axis=1,how='all')

df_pm =(
        pd.merge(df_pm0, df_pm1, how='outer',left_index=True, right_index=True)
        .pipe(lambda d:pd.merge(d, df_pm2, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm3, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm4, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm5, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm6, how='outer',left_index=True, right_index=True))
)

In [5]:
def convert_string_to_tuple_list(string_list):
    tuple_list = []
    for string in string_list:
        # Remove leading and trailing whitespace and parentheses
        string = string.strip('()')
        # Convert string to tuple using ast.literal_eval
        tuple_value = ast.literal_eval(string)
        tuple_list.append(tuple_value)
    return tuple_list

with open("saved_turkey_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_turkey_lat_lon_cords = pickle.load(fp)

with open("saved_turkey_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_turkey_lat_lon_idx = pickle.load(fp)
    
with open("saved_greece_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_greece_lat_lon_cords= pickle.load(fp)

with open("saved_greece_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_greece_lat_lon_idx = pickle.load(fp)
    
saved_turkey_lat_lon_cords = convert_string_to_tuple_list(saved_turkey_lat_lon_cords)
saved_turkey_lat_lon_cords.append((35.0,27.0))
saved_turkey_lat_lon_cords.append((35.0, 45.5))

with open("saved_turkey_and_greece_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_turkey_and_greece_lat_lon_cords= pickle.load(fp)

with open("saved_turkey_and_greece_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_turkey_and_greece_lat_lon_idx = pickle.load(fp)

with open("grecee_pixels_stations_dict_subset.pkl", "rb") as fp:   # Unpickling
    greecee_subset_stations_dict = pickle.load(fp)
with open("grecee_pixels_stations_dict_subset_NN_stations.pkl", "rb") as fp:   # Unpickling
    greecee_subset_stations_dict = pickle.load(fp)
with open("stations_to_avoid_in_case_of_clash.json", "rb") as fp:   # Unpickling
    bad_stations_dict = json.load(fp)
    
    

In [5]:
turkey_subset_stations_dict.pop('(34.5, 33.0)')

[28218]

In [6]:
new_keys = []
for x in list(greecee_subset_stations_dict.keys()):
    x = x[1:-1]
    l = x.split()
    lat = float(l[0])
    lon = float(l[1])
    new_keys.append((lat, lon))

In [7]:
new_dict = {new_key: greecee_subset_stations_dict[old_key] for old_key, new_key in zip(greecee_subset_stations_dict.keys(), new_keys)}


In [8]:
#ls /home/labs/rudich/Collaboration/dust_prediction/data/interpolated_mean_normalized/

# Dnote:



colab_dir = "/home/labs/rudich/Collaboration/dust_prediction/data"
meteo_dir = colab_dir+"/interpolated_mean_normalized/"
meteo_de_dir = colab_dir+"/meteorology_renormalization_tensors"
dust_dir = colab_dir+"/dust_61368_108_2_339/"

pm10_all = torch.load(meteo_de_dir+"/meteorology_tensors_1_81_189_interpolated_mean_normalized_pm10_tensor_denormalized.pkl")
labels_all = torch.load(dust_dir+"dust_61368_108_2_339_full_tensor.pkl")

timestamps_labels = torch.load(dust_dir+"dust_61368_108_2_339_full_timestamps.pkl")
timestamps_mpm = pd.to_datetime(torch.load(colab_dir+"/meteorology_tensors_1_81_189_general_timestamps.pkl"))
timestamps_all = pd.to_datetime(torch.load(colab_dir+"/meteorology_tensors_1_81_189_general_timestamps.pkl"))




In [9]:
colab_dir = "/home/labs/rudich/Collaboration/dust_prediction/data"
meteo_dir = colab_dir+"/interpolated_mean_normalized/"
meteo_de_dir = colab_dir+"/meteorology_renormalization_tensors"
dust_dir = colab_dir+"/dust_61368_108_2_339/"

meteo_meta = torch.load(meteo_dir+"/meteorology_tensor_62_81_189_metadata.pkl")
dust_meta = torch.load(dust_dir+"/dust_61368_108_2_339_metadata.pkl")

lons_dict = meteo_meta['dims']['lons']
lats_dict = meteo_meta['dims']['lats']



In [10]:
df_pm0 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_0.csv",index_col=0).dropna(axis=1,how='all')
df_pm1 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_1.csv",index_col=0).dropna(axis=1,how='all')
df_pm2 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_2.csv",index_col=0).dropna(axis=1,how='all')
df_pm3 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_3.csv",index_col=0).dropna(axis=1,how='all')
df_pm4 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_4.csv",index_col=0).dropna(axis=1,how='all')
df_pm5 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_5.csv",index_col=0).dropna(axis=1,how='all')
df_pm6 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_6.csv",index_col=0).dropna(axis=1,how='all')

df_pm =(
        pd.merge(df_pm0, df_pm1, how='outer',left_index=True, right_index=True)
        .pipe(lambda d:pd.merge(d, df_pm2, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm3, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm4, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm5, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm6, how='outer',left_index=True, right_index=True))


)
df_cord = pd.read_csv("/home/labs/rudich/sagima/TOAR/coordinates_pm.csv")
df_cord = df_cord.rename(columns={df_cord.columns[0]:'station_id'})


first_year = int(df_pm.index[0][:4])
last_year = int(df_pm.index[-1][:4])

timestamps = df_pm.index.values.astype('datetime64[s]').astype(np.int64)
stn_ids = df_pm.columns.values.astype(np.int64)


pm10_values = df_pm.to_numpy()

pm10_tensor = pm10_values.reshape(len(timestamps), len(stn_ids), 1)

# Convert the tensor to a PyTorch tensor
labels_all = torch.from_numpy(pm10_tensor)

timestamp_labels = pd.to_datetime(df_pm.index)

zero_var = (pm10_all.var((2,3))==0).reshape(-1).numpy()
pm10_valid_timestamps = timestamps_all[~zero_var]
pm10_valid_all = pm10_all[~zero_var]

l = [pm10_valid_timestamps, timestamp_labels]
shared_timestamps = l[0].intersection(l[1])

shared_timestamps_idxs = []
for timestamps_list in tqdm(l):
    shared_idxs_per_list = []
    for shared_t in shared_timestamps:
        try:
            shared_idxs_per_list.append(timestamps_list.get_loc(shared_t))
            
        except:
            print(f"Error! Something wierd happend with {shared_t} and {timestamps_list}. Aborting")
        if shared_t == shared_timestamps[-1]:
            print(max(shared_idxs_per_list))
    shared_timestamps_idxs.append(shared_idxs_per_list)

inputs_all = pm10_valid_all
labels_all = labels_all



years = [ i for i in range(first_year, last_year)]
months = [i for i in range(1,13)]

dt = pd.Series(shared_timestamps).dt
years = np.isin(dt.year, years)
months = np.isin(dt.month, months)
include = (years) * (months)

pm10_shared_timestamps = shared_timestamps[include]
inputs = inputs_all[shared_timestamps_idxs[0]][include]
labels = labels_all[shared_timestamps_idxs[1]][include]



/tmp/ipykernel_29387/2475775844.py:26: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamps = df_pm.index.values.astype('datetime64[s]').astype(np.int64)
 50%|██████████████████████████████████████████▌                                          | 1/2 [00:00<00:00,  1.72it/s]

52513


100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.38it/s]

61367


In [11]:
channels46 = ["Q250","Q500","Q700","Q850","Q900",
            "T250","T500","T700","T850","T900",
            "U250","U500","U700","U850","U900",
            "V250","V500","V700","V850","V900",
            "Z250","Z500","Z700","Z850","Z900",
            "PV250","PV500","PV700","PV850","PV900",
            "OMEGA250","OMEGA500","OMEGA700","OMEGA850","OMEGA900",
            "SLP","u10","v10","duaod550",'aermr06_50','aermr06_40','aermr06_30', 'aermr06_20',
              "pm2p5","pm10","tcwv"]


In [12]:
# Define the slice indices
lat_start, lat_end = 10, 75
lon_start, lon_end = 50, 117

# Create new dictionaries for the sliced lat and lon indices
new_lats_dict = {i: lats_dict[i] for i in range(lat_start, lat_end)}
new_lons_dict = {i: lons_dict[i] for i in range(lon_start, lon_end)}

def replace_keys_with_sequential_integers(dictionary):
    new_dict = {}
    keys = list(dictionary.keys())
    for i in range(len(keys)):
        new_dict[i] = dictionary[keys[i]]
    return new_dict

new_lats_dict = replace_keys_with_sequential_integers(new_lats_dict)
new_lons_dict = replace_keys_with_sequential_integers(new_lons_dict)



In [12]:

tensors_list = []
pixels_list = []
#labels_all = torch.load(dust_dir+"dust_61368_108_2_339_full_tensor.pkl")
for channel in tqdm(range(len(channels46))):
    data_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/interpolated_mean_normalized/"
    dust_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/dust_61368_108_2_339/"
    channel_all = torch.load(f"/home/labs/rudich/Collaboration/dust_prediction/data/meteorology_renormalization_tensors/meteorology_tensors_1_81_189_interpolated_mean_normalized_{channels46[channel]}_tensor_denormalized.pkl")
    subset_channel = channel_all[:, :, lat_start:lat_end, lon_start:lon_end]
    tensors_list.append(subset_channel)
stacked_tensor = torch.stack(tensors_list, dim=1)
stacked_tensor = stacked_tensor.squeeze(dim=2)
stacked_tensor = stacked_tensor[~zero_var]
print(stacked_tensor.shape)

100%|███████████████████████████████████████████████████████████████████████████████████| 46/46 [01:39<00:00,  2.15s/it]


torch.Size([52514, 46, 15, 37])


In [13]:
all_stations_lonlat = (df_cord 
                       .rename(columns={'lng':'lon'})
                       .assign(stn_id=np.arange(0,df_cord.shape[0],1))
                        )
station_df = all_stations_lonlat
pm10_df = df_pm



In [14]:

stations_tensor = np.zeros((pm10_valid_timestamps.shape[0], 1, len(new_lats_dict), len(new_lons_dict)))

inv_lats = {v: k for k, v in new_lats_dict.items()}
inv_lons = {v: k for k, v in new_lons_dict.items()}
list_of_pixels_with_stations = []

In [15]:
pm10_df.index = pd.to_datetime(pm10_df.index)
pm10_df.columns =  pm10_df.columns.astype(int)
set1 = set(pm10_valid_timestamps)
set2 = set(pm10_df.index)

# Find the intersection of the sets
intersection = set1.intersection(set2)

# Convert the intersection set to a DatetimeIndex
common_timestamps = pd.DatetimeIndex(pd.to_datetime(list(intersection)))
common_timestamps = common_timestamps.sort_values()
# Find the index where the intersection starts
start_index = list(pm10_valid_timestamps).index(common_timestamps[0])

# Find the index where the intersection ends
end_index = list(pm10_valid_timestamps).index(common_timestamps[-1])


In [19]:
new_dict.pop((41.5, 26.5))
new_dict.pop((41.0, 26.5))
new_dict.pop((40.0, 26.5))

[56722]

# Creating a dict with pixel coordinates - stations id - distances 

In [20]:
stations_tensor[:, 0, :, :] = np.nan

mean_values = np.zeros(stations_tensor.shape[:3])


for key in tqdm(new_dict):
    lat = key[0]
    lon = key[1]

    lat_index = inv_lats[lat]
    lon_index = inv_lons[lon]

    df_rows = pm10_df.loc[common_timestamps, new_dict[key]]

    mean_values = df_rows.mean(axis=1) 

    stations_tensor[:, 0, lat_index, lon_index] = mean_values


100%|███████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 59.86it/s]


In [58]:
stations_tensor[:,0,:,:] = np.nan
l = list(bad_stations_dict.values())
bad_stations = [item for sublist in l for item in sublist]

for key in tqdm(turkey_subset_stations_dict):

    lat = key[0]
    lon = key[1]

    # Get the corresponding indices from the inv_lats and inv_lons dictionaries
    lat_index = inv_lats[lat]
    lon_index = inv_lons[lon]

    for t, timestamp in enumerate(common_timestamps):
        if t >= start_index:
            
            df_row = pm10_df.loc[timestamp]
            series_helper = df_row.loc[turkey_subset_stations_dict[key]].dropna()
            filtered_df = df_row[[element for element in series_helper.index if element not in bad_stations]]
            mean_value = filtered_df.mean()
            stations_tensor[t, 0, lat_index, lon_index] = mean_value


100%|███████████████████████████████████████| 118/118 [1:11:23<00:00, 36.30s/it]


In [104]:
# mask_check = np.isnan(tensor_check)
# mask_stations = np.isnan(stations_tensor)

# # Replace NaN values with zeros (or any other value)
# tensor_check[mask_check] = 0
# stations_tensor[mask_stations] = 0

# # Calculate the Euclidean distance
# euclidean_distance = np.linalg.norm(tensor_check - stations_tensor)

In [111]:
# stations_tensor = torch.load('stations_fixed3.pt')
# torch.save(torch.from_numpy(stations_tensor), 'stations_fixed3.pt')
# full = torch.cat((stacked_tensor,torch.from_numpy(stations_tensor)),dim=1)
# full = torch.cat((stacked_tensor,stations_tensor),dim=1)
# del stacked_tensor
# del stations_tensor

In [5]:
# torch.save(sliced_tensor, 'new_turkey_greece_tensor_fixed.pt')
# sliced_tensor = torch.load('new_turkey_greece_tensor_fixed.pt')

In [60]:
torch.save(torch.tensor(stations_tensor),'turkey_stations_tensor_subset_ver3.pt')

In [17]:
stations_tensor = torch.load('turkey_stations_tensor_subset.pt')
full = torch.cat((stacked_tensor,stations_tensor),dim=1)


In [18]:
del stacked_tensor
del stations_tensor

In [113]:
full = torch.cat((stacked_tensor,torch.tensor(stations_tensor)),dim=1)
del stacked_tensor
del stations_tensor

In [114]:
torch.save(full, 'turkey_subset_stations_full_tensor_ver2.pt')

In [28]:
full = torch.load('turkey_subset_stations_full_tensor_ver2.pt')

In [19]:
full.shape

torch.Size([52514, 47, 15, 37])

In [42]:
stations_tensor.shape

(52514, 1, 65, 67)

# tensor to dataframe

In [21]:

tensor = stations_tensor

tensor_shape = (52514, 1, 65, 67)

timestamps = common_timestamps # List of timestamps (length 52514)


reshaped_tensor = np.reshape(tensor, (tensor_shape[0], tensor_shape[1], -1))

dfs = []

for lat_index in tqdm(range(tensor_shape[2])):
    for lon_index in range(tensor_shape[3]):        

        df = pd.DataFrame(reshaped_tensor[:, :, (lat_index * tensor_shape[3]) + lon_index])

        df['time'] = timestamps
        lat_loc = new_lats_dict[lat_index]
        lon_loc = new_lons_dict[lon_index]
        df['lat_lon_coord'] = str((float(lat_loc),float(lon_loc)))

        dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
del dfs
del tensor
# del full

100%|███████████████████████████████████████████████████████████████████████████████████| 65/65 [00:06<00:00,  9.38it/s]


In [22]:
df = df.set_axis(['stn_mesm','time','lat_lon_coord'],axis=1)

In [23]:
df.to_parquet('plot_helper_nn_stations.parquet.gzip', compression='gzip')

In [67]:
df[[0]].to_parquet('fixed_stations_column.gzip', compression='gzip')

In [33]:

df = (
       df
      .set_axis(channels46 +['stn_mesm','time','lat_lon_coord'],axis=1 )
      .drop('pm2p5',axis=1)
     )


In [34]:
grouped = df.groupby('lat_lon_coord')

# Creating new columns with previous row values for each group
for column in df.drop(['stn_mesm','time','lat_lon_coord'],axis=1).columns:
    new_column_name = column + '_prev'
    df[new_column_name] = grouped[column].shift()
    
df['week'] = df['time'].dt.strftime('%W')
df['hour'] = df['time'].dt.hour
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year
df['sin'] = np.sin(df['time'].dt.dayofyear * 2 * np.pi / 365)
df['cos'] = np.cos(df['time'].dt.dayofyear * 2 * np.pi / 365)



In [23]:
df.shape

(29145270, 49)

In [35]:
def add_delta_columns(df):
    channels_used =  ["Q250","Q500","Q700","Q850","Q900",
            "T250","T500","T700","T850","T900",
            "U250","U500","U700","U850","U900",
            "V250","V500","V700","V850","V900",
            "Z250","Z500","Z700","Z850","Z900",
            "PV250","PV500","PV700","PV850","PV900",
            "OMEGA250","OMEGA500","OMEGA700","OMEGA850","OMEGA900",
            "SLP","u10","v10","duaod550",'aermr06_50','aermr06_40','aermr06_30', 'aermr06_20',
              "tcwv"]

    # Loop over the channels names
    for col in channels_used:
        
        # Calculate the delta between the column and its corresponding "_prev" column
        delta_col = col + '_delta'
        prev_col = col + '_prev'
        df[delta_col] = df[col] - df[prev_col]
    
    return df

df = add_delta_columns(df)
df = df.drop(['pm10_prev'],axis=1)
df['pm10'] = df['pm10'] * (10**9)
df['target'] = df['stn_mesm'] - df['pm10']

In [36]:
df.head(2)

,Q250,Q500,Q700,Q850,Q900,T250,T500,T700,T850,T900,...,SLP_delta,u10_delta,v10_delta,duaod550_delta,aermr06_50_delta,aermr06_40_delta,aermr06_30_delta,aermr06_20_delta,tcwv_delta,target
0,0.000016,0.000148,0.002609,0.005352,0.006493,220.551987,256.688263,272.292419,279.520905,282.570587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-16.577514
1,0.000013,0.000539,0.001013,0.003010,0.004505,219.105560,255.949677,272.005768,278.810516,282.191406,...,-1.328796,0.420458,0.003151,0.02826,5.319999e-09,3.791060e-11,-3.630541e-12,-1.508498e-12,-1.56883,-32.734256


In [44]:
t = df['time'].sort_values().unique()


In [46]:
with open('timestamps_turkey_subset.pkl','wb') as f:
    pickle.dump(t,f)

In [37]:
a =( 
    pd.read_parquet('turkey_ndvi_subset_stations.parquet.gzip')
    .set_axis(['NDVI', 'time','lat_lon_coord'],axis=1)
)

In [38]:
df.shape, a.shape

((29145270, 143), (29145270, 3))

In [39]:
df['NDVI'] = a['NDVI']

In [40]:
df.to_parquet('df_turkey_subset_stations_ver2.parquet.gzip',
              compression='gzip')

In [54]:
# del sliced_tensor
# del tensor
# df.to_csv('turkey_greece_fixed.csv',index=False)